# APDFT Analysis

Predictions can use

- **QC**: HF, CCSD, or CCSD(T) energy differences with the species of interests or target,
- **Quantum Alchemy**: HF, CCSD, or CCSD(T) energy differences by changing the nuclear charge of other systems that have the same number of electrons as the target,
- **APDFT*n***: Taylor series approximation with order *n* of the APDFT potential energy surface.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors

from apdft_tools.utils import hartree_to_ev, all_atom_systems
from apdft_tools.data import prepare_dfs
from apdft_tools.prediction import get_apdft_change_charge, get_qc_change_charge
from apdft_tools.analysis import apdft_error_change_charge

json_path = '../../json-data/atom-pyscf.apdft-data.posthf.json'
df_qc, df_apdft = prepare_dfs(json_path, get_CBS=False)

## APDFT prediction errors

There is some intrinsic error in modeling a target system (e.g., N atom) by changing the nuclear charge of a reference system's basis set (e.g., C<sup> &ndash;</sup> ).
The following cell computes this error, and represents the best performance for APDFT without fortuitous errors.

In [2]:
system_label = 'n'
delta_charge = 1
target_initial_charge = 0  # Initial charge of the system.
basis_set = 'aug-cc-pV5Z'  # cc-pV5Z, aug-cc-pVTZ, aug-cc-pVQZ, aug-cc-pV5Z, CBS-aug

use_fin_diff = False  # Use finite differences with Taylor series for APDFT predictions.
change_signs = False  # Multiple all predictions by negative one (e.g., for electron affinities)

ie_qc_prediction = get_qc_change_charge(
    df_qc, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set
)
ie_apdft_predictions = get_apdft_change_charge(
    df_qc, df_apdft, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set, use_fin_diff=use_fin_diff
)

ie_qc_prediction = hartree_to_ev(ie_qc_prediction)
ie_apdft_predictions = {key:hartree_to_ev(value) for (key,value) in ie_apdft_predictions.items()}
ie_apdft_errors = {key:value-ie_qc_prediction for (key,value) in ie_apdft_predictions.items()}

print(f'PySCF prediction of IE for {system_label}: {ie_qc_prediction:.3f} eV\n')
print(f'APDFT prediction errors in eV:')
print(pd.DataFrame(ie_apdft_errors, index=[f'APDFT']))

PySCF prediction of IE for n: 14.541 eV

APDFT prediction errors in eV:
              b         c         o
APDFT -0.053009 -0.010871  0.001612


## APDFT*n* prediction errors

Now, we can look at approximating the APDFT prediction by using a Taylor series centered on $\Delta Z = 0$.

In [3]:
system_label = 'n'
delta_charge = 1
target_initial_charge = 0  # Initial charge of the system.
basis_set = 'aug-cc-pV5Z'  # cc-pV5Z, aug-cc-pVTZ, aug-cc-pVQZ, aug-cc-pV5Z, CBS-aug

use_fin_diff = True  # Use finite differences with Taylor series for APDFT predictions.
change_signs = False  # Multiple all predictions by negative one (e.g., for electron affinities)

ie_qc_prediction = get_qc_change_charge(
    df_qc, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set
)
ie_apdft_predictions = get_apdft_change_charge(
    df_qc, df_apdft, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set, use_fin_diff=use_fin_diff
)

ie_qc_prediction = hartree_to_ev(ie_qc_prediction)
ie_apdft_predictions = {key:hartree_to_ev(value) for (key,value) in ie_apdft_predictions.items()}
ie_apdft_errors = {key:value-ie_qc_prediction for (key,value) in ie_apdft_predictions.items()}

print(f'PySCF prediction of IE for {system_label}: {ie_qc_prediction:.3f} eV\n')
print(f'APDFTn prediction errors in eV:')
print(pd.DataFrame(ie_apdft_errors, index=[f'APDFT{n}' for n in range(5)]))

PySCF prediction of IE for n: 14.541 eV

APDFTn prediction errors in eV:
                b          c          o
APDFT0 -17.880868 -13.288335  20.598492
APDFT1 -14.107617  -4.001721  -3.519578
APDFT2 -14.026723   0.459099  -0.030341
APDFT3   9.209186  -0.214531  -0.005005
APDFT4  45.454693  -0.470333   0.000159


### Specifying lambda values

We can also specify specific lambda values to include. For example, we could only look at lambda values of +-1.

In [4]:
system_label = 'n'
delta_charge = 1
target_initial_charge = 0  # Initial charge of the system.
basis_set = 'aug-cc-pV5Z'  # cc-pV5Z, aug-cc-pVTZ, aug-cc-pVQZ, aug-cc-pV5Z, CBS-aug

considered_lambdas = [-1, 1]

use_fin_diff = True  # Use finite differences with Taylor series for APDFT predictions.
change_signs = False  # Multiple all predictions by negative one (e.g., for electron affinities)

ie_qc_prediction = get_qc_change_charge(
    df_qc, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set
)
ie_apdft_predictions = get_apdft_change_charge(
    df_qc, df_apdft, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set, use_fin_diff=use_fin_diff,
    considered_lambdas=considered_lambdas
)

ie_qc_prediction = hartree_to_ev(ie_qc_prediction)
ie_apdft_predictions = {key:hartree_to_ev(value) for (key,value) in ie_apdft_predictions.items()}
ie_apdft_errors = {key:value-ie_qc_prediction for (key,value) in ie_apdft_predictions.items()}

print(f'PySCF prediction of IE for {system_label}: {ie_qc_prediction:.3f} eV\n')
print(f'APDFTn prediction errors in eV:')
print(pd.DataFrame(ie_apdft_errors, index=[f'APDFT{n}' for n in range(5)]))

PySCF prediction of IE for n: 14.541 eV

APDFTn prediction errors in eV:
                c          o
APDFT0 -13.288335  20.598492
APDFT1  -4.001721  -3.519578
APDFT2   0.459099  -0.030341
APDFT3  -0.214531  -0.005005
APDFT4  -0.470333   0.000159


## APDFT*n* errors with respect to APDFT

Or you, can compute the difference between APDFT*n* (predictions with Taylor series) and APDFT.

In [5]:
system_label = 'n'
delta_charge = 1
target_initial_charge = 0  # Initial charge of the system.
basis_set = 'aug-cc-pV5Z'  # cc-pV5Z, aug-cc-pVTZ, aug-cc-pVQZ, aug-cc-pV5Z, CBS-aug

compute_difference = True  # Returns APDFTn - APDFT instead of energy predictions.

use_fin_diff = True  # Use finite differences with Taylor series for APDFT predictions.
change_signs = False  # Multiple all predictions by negative one (e.g., for electron affinities)

ie_apdft_predictions = get_apdft_change_charge(
    df_qc, df_apdft, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set, use_fin_diff=use_fin_diff,
    compute_difference=compute_difference
)

ie_apdft_predictions = {key:hartree_to_ev(value) for (key,value) in ie_apdft_predictions.items()}

print(f'Differences between APDFTn and APDFT in eV:')
print(pd.DataFrame(ie_apdft_predictions, index=[f'APDFT{n}' for n in range(5)]))

Differences between APDFTn and APDFT in eV:
                b          c          o
APDFT0 -17.827859 -13.277465  20.596880
APDFT1 -14.054608  -3.990850  -3.521190
APDFT2 -13.973714   0.469970  -0.031953
APDFT3   9.262195  -0.203660  -0.006618
APDFT4  45.507701  -0.459463  -0.001454


## Overall statistics

We can also compute mean absolute errors (MAEs), root mean squared error (RMSE) and max error.

In [6]:
all_systems = all_atom_systems[1:]
basis_set = 'aug-cc-pV5Z'
target_initial_charge = 0

use_fin_diff = True
compute_difference = False
considered_lambdas = [-1, 1]

delta_charge = 1
change_signs = False
max_apdft_order = 4
ignore_one_row = True


for i in range(len(all_systems)):
    sys_error = apdft_error_change_charge(
        df_qc, df_apdft, all_systems[i], delta_charge, change_signs=change_signs, 
        basis_set=basis_set, target_initial_charge=target_initial_charge,
        use_fin_diff=use_fin_diff, ignore_one_row=ignore_one_row,
        compute_difference=compute_difference,
        considered_lambdas=considered_lambdas
    )
    if i == 0:
        all_error = sys_error
    else:
        all_error = pd.concat(
            [all_error, sys_error], axis=1
        )

if use_fin_diff or compute_difference == True:
    # MAE
    for n in range(0, max_apdft_order+1):
        apdftn_errors = all_error.iloc[n].values
        apdftn_mae = np.mean(np.abs(apdftn_errors))
        print(f'APDFT{n} MAE: {apdftn_mae:.4f} eV')

    # RMSE
    print()
    for n in range(0, max_apdft_order+1):
        apdftn_errors = all_error.iloc[n].values
        apdftn_rmse = np.sqrt(np.mean((apdftn_errors)**2))
        print(f'APDFT{n} RMSE: {apdftn_rmse:.4f} eV')
    
    # Max
    print()
    for n in range(0, max_apdft_order+1):
        apdftn_errors = all_error.iloc[n].values
        apdftn_max = np.max(np.abs(apdftn_errors))
        print(f'APDFT{n} Max Abs.: {apdftn_max:.4f} eV')
else:
    # MAE
    apdftn_errors = all_error.iloc[0].values
    apdftn_mae = np.mean(np.abs(apdftn_errors))
    print(f'APDFT MAE: {apdftn_mae:.4f} eV')

    # RMSE
    print()
    apdftn_rmse = np.sqrt(np.mean((apdftn_errors)**2))
    print(f'APDFT RMSE: {apdftn_rmse:.4f} eV')

    # Max
    print()
    apdftn_max = np.max(np.abs(apdftn_errors))
    print(f'APDFT Max Abs.: {apdftn_max:.4f} eV')


APDFT0 MAE: 14.5811 eV
APDFT1 MAE: 3.6138 eV
APDFT2 MAE: 0.3958 eV
APDFT3 MAE: 1.0075 eV
APDFT4 MAE: 2.7050 eV

APDFT0 RMSE: 16.6775 eV
APDFT1 RMSE: 4.5460 eV
APDFT2 RMSE: 0.6555 eV
APDFT3 RMSE: 2.7752 eV
APDFT4 RMSE: 6.5716 eV

APDFT0 Max Abs.: 50.5211 eV
APDFT1 Max Abs.: 13.7508 eV
APDFT2 Max Abs.: 2.7449 eV
APDFT3 Max Abs.: 12.9765 eV
APDFT4 Max Abs.: 25.9311 eV
